In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import random
import pandas as pd
from simanneal import Annealer
import nest
from scripts import optimization
import time


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.6.0
 Built: Sep 28 2023 11:53:34

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



In [2]:
nest.rng_seed = 1
np.random.seed(1)
nest.set_verbosity('M_FATAL')

In [3]:
experimental_data = np.load('/hpc/mzhu843/modelling/nest/DATA/processed/experiment3_unfiltered_processed.npy')
initial_connectivity_matrix = np.load('/hpc/mzhu843/modelling/nest/DATA/matrices/matrix.npy')
lamb = 0.01

In [4]:
class OptimizationProblem(Annealer):
    def __init__(self, state, data, lamb):
        self.experimental_data = data
        self.lamb = lamb
        self.state = state

    def energy(self):
        print(self.state[10])
        results = optimization.simulate(self.state)
        results = optimization.simulation_results_to_spike_trains(results, 1) #Has to be the same as the script runtime!
        cost_function = optimization.ssd_with_l1(results, experimental_data, lamb)
        print(cost_function)
        return cost_function

    def move(self):
        print(self.state[1][1])
        for i in range(20000):
            x = np.random.randint(0, 276)
            y = np.random.randint(0, 276)
            if self.state[x][y] != 0:
                w = np.random.randint(1, 20)
                self.state[x][y] = w

In [5]:
problem = OptimizationProblem(initial_connectivity_matrix, experimental_data, 0.01)
x, func = problem.anneal()

[1.15494148 0.06015858 1.13902563 0.88813552 0.75531153 1.0856491
 0.98286106 1.20049142 0.88911156 1.41997581 0.5062178  1.10673312
 1.42366701 1.0693211  1.46981082 0.82417631 0.95483729 1.05316042
 0.28573107 0.41326212 0.65014175 0.59949875 0.91375791 1.3028768
 0.04510813 1.4634405  1.16490024 1.18144424 0.00566454 1.0967768
 0.81747815 1.15969297 1.18008612 1.79686907 1.48674305 0.75407104
 0.81585867 2.01150249 0.80749204 0.29990382 0.89298903 1.36730841
 0.99502435 1.47190414 0.71644134 1.23202272 0.33172781 0.59813775
 1.04385727 1.23704434 0.68767697 0.54668197 1.52636184 1.51404045
 1.27729965 1.25987901 0.87025445 1.41007008 0.7185097  1.02338785
 1.36274599 1.58410032 1.11372252 0.78255281 1.09126572 1.90190787
 0.83557869 0.30268057 0.91225824 0.60297009 1.03968244 1.09465344
 1.61867781 1.73148761 1.12378989 1.21190624 1.16694464 2.17093251
 1.22072542 0.93354565 0.47926954 0.89293077 1.11574836 1.03765641
 1.15825528 0.71123668 0.24197436 1.07520243 1.04083489 1.3177125

 Temperature        Energy    Accept   Improve     Elapsed   Remaining


10316.0
0.34539392277027303
[1.15494148e+00 7.00000000e+00 1.13902563e+00 1.10000000e+01
 7.55311530e-01 1.08564910e+00 9.82861061e-01 1.20049142e+00
 8.89111561e-01 1.41997581e+00 5.06217800e-01 1.10000000e+01
 1.42366701e+00 1.06932110e+00 3.00000000e+00 8.24176312e-01
 9.54837294e-01 1.05316042e+00 2.85731068e-01 4.13262122e-01
 6.50141747e-01 5.99498748e-01 9.13757915e-01 1.90000000e+01
 4.51081313e-02 1.46344050e+00 4.00000000e+00 1.18144424e+00
 1.00000000e+01 1.09677680e+00 8.17478152e-01 1.15969297e+00
 1.18008612e+00 2.00000000e+00 1.48674305e+00 7.54071039e-01
 8.15858674e-01 1.40000000e+01 8.07492036e-01 2.99903816e-01
 4.00000000e+00 1.36730841e+00 1.00000000e+01 1.47190414e+00
 7.16441336e-01 1.30000000e+01 1.80000000e+01 5.98137754e-01
 5.00000000e+00 1.23704434e+00 9.00000000e+00 5.46681970e-01
 1.52636184e+00 1.51404045e+00 1.27729965e+00 1.25987901e+00
 3.00000000e+00 1.41007008e+00 1.00000000e+00 1.02338785e+00
 1.36274599e+00 1.58410032e+00 1.11372252e+00 7.82552814e

KeyboardInterrupt: 